In [1]:
from ml.features.preprocessing import get_data
from ml.ml_functions.registry.model_registry import log_sklearn_model_to_mlflow
from ml.ml_functions.serve.mlflow_model_serve import mlflow_model_prediction

## Load Dataset with new Features

In [4]:
stock_data, last_day_df = get_data(save_data=False)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

In [9]:
last_day_df

,Close,Volume,SMA 10,EMA 10,EMA 20,WMA 10,Momentum 10,SAR,RSI,ROC,...,^IBEX Close,SI=F Close,HG=F Close,NG=F Close,^TNX Close,^IRX Close,^FVX Close,^TYX Close,SPY Close,EFA Close
Date,,,,,,,,,,,,,,,,,,,,,
2023-12-04,4569.779785,4.369910e+09,4558.963965,4547.842847,4495.799442,4564.698127,55.759766,4537.240234,67.561481,1.235257,...,10178.299805,24.555,3.8205,2.694,4.288,5.223,4.238,4.438,456.690002,72.610001


## Train your Sklearn-Model

In [4]:
from ml.ml_functions.training.train_extra_tree import train_extra_tree

model, accuracy = train_extra_tree(stock_data)

Modellname: ExtraTreesClassifier
Genauigkeit: 0.9048027444253859


## Model Registry 

### mlflow server --host 0.0.0.0 --port 5000


In [11]:
actual_model_path = log_sklearn_model_to_mlflow(model=model, accuracy=accuracy)

2023/12/05 10:56:46 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: best_ExtraTreesClassifier_model, version 1


## Save current feature data for prediction

In [3]:
stock_data, last_day_df = get_data(save_data=True, new_model=(model.__class__.__name__, accuracy))

NameError: name 'model' is not defined

### Predict the new Data with Mlflow-Model

In [2]:
result = mlflow_model_prediction(model=actual_model_path)

/home/erik/anaconda3/envs/Stock-Prediction/lib/python3.8/site-packages/sklearn/base.py:458: UserWarning: X has feature names, but ExtraTreesClassifier was fitted without feature names
  warnings.warn(


Complete. Return to the InfluxDB UI.
